<a href="https://colab.research.google.com/github/shreyammb/INNOV8TIGERS/blob/main/welfare_placement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy==1.26.4
!pip install rasterio scikit-learn-extra

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.4.2
    Uninstalling numpy-2.4.2:
      Successfully uninstalled numpy-2.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
pytensor 2.37.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.13.0.90 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.13.0.90 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is 

  Using cached numpy-2.4.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)
Using cached numpy-2.4.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.4.2 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.4.2 which is incompatible.


In [1]:
from google.colab import files

uploaded = files.upload()
INPUT_TIF = list(uploaded.keys())[0]

Saving pred_LightGBM (1) (3).tif to pred_LightGBM (1) (3) (3).tif


In [2]:
import rasterio
import numpy as np

with rasterio.open(INPUT_TIF) as src:
    slum_mask = src.read(1)
    transform = src.transform
    crs = src.crs
    profile = src.profile

rows, cols = np.where(slum_mask == 1)

print(f"Total slum pixels: {len(rows)}")


Total slum pixels: 186705


In [3]:
coords = np.array([
    rasterio.transform.xy(transform, r, c)
    for r, c in zip(rows, cols)
])


In [5]:
from sklearn.cluster import KMeans

K = 500  # no of facilities

kmeans = KMeans(
    n_clusters=K,
    random_state=42,
    n_init=10
)

kmeans.fit(coords)

facility_coords = kmeans.cluster_centers_

from scipy.spatial import cKDTree

tree = cKDTree(coords)
_, idx = tree.query(facility_coords)

facility_coords = coords[idx]


In [6]:
facility_pixels = [
    rasterio.transform.rowcol(transform, x, y)
    for x, y in facility_coords
]


In [7]:
output = np.zeros_like(slum_mask, dtype=np.uint8)

for r, c in facility_pixels:
    if 0 <= r < output.shape[0] and 0 <= c < output.shape[1]:
        output[r, c] = 1


In [8]:
profile.update(dtype=rasterio.uint8, count=1)

OUTPUT_TIF = "facility_locations.tif"

with rasterio.open(OUTPUT_TIF, "w", **profile) as dst:
    dst.write(output, 1)

print("Saved:", OUTPUT_TIF)


Saved: facility_locations.tif


In [9]:
files.download(OUTPUT_TIF)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
import rasterio
import numpy as np

with rasterio.open(OUTPUT_TIF) as src:
    facility_raster = src.read(1)

num_facilities = np.sum(facility_raster == 1)

print("Sanity Check Results")
print(f"Expected number of facilities (K): {K}")
print(f"Actual number of facility pixels : {num_facilities}")

if num_facilities == K:
    print("Sanity check passed: counts match")
else:
    print("Sanity check failed: mismatch detected")


Sanity Check Results
--------------------
Expected number of facilities (K): 50
Actual number of facility pixels : 50
✅ Sanity check passed: counts match
